<a href="https://colab.research.google.com/github/joeywangyolo/cookbot/blob/main/linebottest_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError,
    LineBotApiError
)
from linebot.models import *

In [ ]:
app = Flask(__name__,static_url_path='/material',static_folder='./material')
run_with_ngrok(app)

line_bot_api = LineBotApi('aaVojFnTBTE7+BYR9Y1+AzjrnOvkIgBAW8QxVeqWMqxI4mQKD7nifTQAaPnnY4BJkPRUz/zPdnV7+KK8tqRCYjLDepPHWug/DNk4dtdyKMe+dqi29x312vvMSim4uMnVR5IFpOaNT+ju2qzkCK1woAdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler("9bdeaaf10e10fc84052ea6b2c268db7f")

In [ ]:
from linebot.http_client import HttpResponse
#LineBot 的主程序對話入口
#啟動server對外接口，使Line能丟消息進來
@app.route("/" , methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text = True)
    app.logger.info("Request body: " + body)
    print(body)

    # 記錄用戶log
    #f = open("/content/drive/MyDrive/ai-event.log", "a")
    #f.write(body)
    #f.close()

    # handle webhook body
    try:
        events=handler.handle(body, signature)
        #handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
from linebot.models import *
from traitlets.traitlets import EventHandler
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    #用戶傳來的文字消息
    #若有 find 關鍵字 則執行下面的方法
    msg = event.message.text
    print(msg)
    if '食譜'in msg :
      fruits_massage = fruits_crawler()
      print(fruits_massage)
      line_bot_api.reply_message(
          event.reply_token,
          #FlexSendMessage('食譜',fruits_massage)
          FlexSendMessage(fruits_massage)
      )
    #若無，回應查無此字
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=" 字典內查無此字，請輸入'食譜' ")
        )

In [ ]:
def fruits_crawler():
  import requests
  from bs4 import BeautifulSoup
  import json
  url = 'https://www.superbuy.com.tw/collection_fast.php?main_cate=610'
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
  myHeaders={'User-Agent': user_agent}
  r=requests.get(url, headers=myHeaders)
  print(r.status_code)
  total =[]
  result =[]
  if r.status_code == 200:
      soup = BeautifulSoup(r.text,'html.parser')
      fruits = soup.find_all(
      'div',{'class':'product col-xs-6 col-sm-6 col-md-4'},
      limit = 5
      )
      #print(fruits)
      print('4')
      for fruits in fruits:
          img = fruits.find_all('li')[0].find('img')['src']
        # print(img)
          name = fruits.find_all('li')[2].text.strip()
        # print(name)
          weight = fruits.find_all('li')[3].text.strip()
          #print(weight)
          price = fruits.find_all('li')[4].text.strip()
          links = fruits.find_all('li')[2].find('a')['href']
          links = "https://www.superbuy.com.tw/"+links
          #print(links)
          total.append({"img":img,
                        "name":name,
                        "weight":weight,
                        "price":price,
                        "links":links
                            })
          print('7')
      with open('/content/foodie.json',encoding='utf-8') as f:
          fruits = json.load(f)
          print('8')
          for i in range(3):
              fruits['contents'][i]['hero']['url'] = total[i]['img']
              fruits['contents'][i]['body']['contents'][0]['text']= total[i]['name']
              fruits["contents"][i]["body"]["contents"][2]["contents"][0]["contents"][0]['text']= total[i]['weight']
              fruits["contents"][i]["footer"]['contents'][0]['action']['uri']= total[i]['links']
          fruits_massage = FlexSendMessage.new_from_json_dict(fruits)   
      with open('/content/foodie.json','w',newline='',encoding='utf-8') as f:
          json.dump(fruits,f,indent=4,ensure_ascii=False)
          #print(fruits['contents'][1]['hero']['url'])
          #print(fruits['contents'][1]['body']['contents'][0]['text'])
      return fruits_massage

In [ ]:
app.run()